In [ ]:
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127283 sha256=1a9b58fa5f78c93615e52efb1d594709ca2d2d86617e526aef5e80ad934e4e25
  Stored in directory: /root/.cache/pip/wheels/55/fb/7b/e06204a0ceefa45443930b9a250cb5ebe31def0e4e8245a465
Successfully built tflearn


In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
import json
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import random
import tflearn
import numpy as np
import pickle
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
print(tf.version)

<module 'tensorflow.compat.v1.version' from '/usr/local/lib/python3.10/dist-packages/tensorflow/_api/v2/compat/v1/version/__init__.py'>


In [ ]:
#tokenize words
with open('Intents.json') as file:
  data=json.load(file)
try:
  with open("data.pickle",'rb') as phile:
      words,labels,Training,Testing=pickle.load(phile)
except:
  words=[]
  labels=[]
  doc_x=[]
  doc_y=[]
  for intent in data['intents']:
    for pattern in intent['patterns']:
       wrds= nltk.word_tokenize(pattern)
       words.extend(wrds)
       doc_x.append(wrds)
       doc_y.append(intent['tag'])
    if intent['tag'] not in labels:
       labels.append(intent['tag'])

  words = [stemmer.stem(w.lower()) for w in words if w != '?' or w != '!']
  words= sorted(list(set(words)))
  labels=sorted(labels)
  #bag of words
  Training=[]
  Testing=[]

  output=[0 for _ in range(len(labels))]

  for x, doc in enumerate(doc_x):
     bag=[]
     wrds=[stemmer.stem(w.lower()) for w in doc]
     for w in words:
       if w in wrds:
         bag.append(1)
       else:
         bag.append(0)

     Testing_row=output[:]
     Testing_row[labels.index(doc_y[x])] = 1

     Training.append(bag)
     Testing.append(Testing_row)

  Training=np.array(Training)
  Testing=np.array(Testing)
  with open("data.pickle","wb") as phile:
    pickle.dump((words,labels,Training,Testing),phile)

FileNotFoundError: ignored

In [ ]:
Training

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [ ]:
tf.reset_default_graph()
network=tflearn.input_data(shape=[None,len(Training[0])])
network=tflearn.fully_connected(network,8)
network=tflearn.fully_connected(network,8)
network=tflearn.fully_connected(network,len(Testing[0]),activation='softmax')
network= tflearn.regression(network)
model=tflearn.DNN(network)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
model.fit(Training, Testing, n_epoch=500, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 1999  | total loss: 0.29187 | time: 0.021s
| Adam | epoch: 500 | loss: 0.29187 - acc: 0.9718 -- iter: 24/25
Training Step: 2000  | total loss: 0.31268 | time: 0.026s
| Adam | epoch: 500 | loss: 0.31268 - acc: 0.9746 -- iter: 25/25
--


In [ ]:
def bag(inp,words):
    s = [0 for _ in range(len(words))]
    t = nltk.word_tokenize(inp)
    t = [stemmer.stem(w.lower()) for w in t]
    for se in t:
        for i,x in enumerate(words):
           if x == se:
              s[i]=1
    return np.array(s)
bot_name='AVA'

In [ ]:
def chat():
    while True:
        inp = input('Warlock: ')
        if inp.lower() == 'quit' or inp.lower() == 'signing off':
           break
        result = model.predict([bag(inp,words)])
        index = np.argmax(result)
        tag = labels[index]
        for tg in data['intents']:
            if tg['tag'] == tag:
              responses = tg['responses']

        print(random.choice(responses))
chat()

Warlock: hey AVA
I am Augumented Virtual Assistant
Warlock: Hey AVA
I am Augumented Virtual Assistant
Warlock: what's up
I'm still learning. Can you provide more information?
Warlock: how are you 
Hi there!
Warlock: signing off
